###  Import libraries


In [5]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from tensorflow import keras
from zipfile import ZipFile
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

### Hiperparámetros

In [6]:
csv_path          = "regina.csv"
predicted_feature = 'Temp Out'
dropped_features  = ['Tpot (K)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'Day cos', 'Year sin']

train_perc = .7
val_perc   = .2

sequence_length = 48
offset          = 5
sampling_rate   = 3
length          = 6 #horas
min_temp        = 100
batch_size      = 256

learning_rate   = 0.001
epochs          = 10
loss            = "mse"

### Read CSV

In [7]:
dateparse = lambda x: datetime.strptime(x, '%d.%m.%y %H:%M')

data = pd.read_csv(csv_path, parse_dates=['Date Time'], date_parser=dateparse, index_col=0)
data.head()

,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Dir,Wind Run,Hi Speed,Hi Dir,...,Cool D-D,In Temp,In Hum,In Dew,In Heat,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int
Date Time,,,,,,,,,,,,,,,,,,,,,
2010-01-28 14:20:00,32.9,33.1,32.9,21,7.7,0.0,---,0.00,0.0,---,...,0.101,33.0,21,7.8,31.2,0.00,227,1,99.6,10
2010-01-28 14:30:00,32.9,33.1,32.7,24,9.7,1.6,ENE,0.27,41.8,NNE,...,0.101,33.2,24,9.9,31.8,0.00,229,1,100.0,10
2010-01-28 14:40:00,30.6,32.6,28.7,28,10.0,0.0,N,0.00,22.5,NNW,...,0.085,32.1,26,10.1,30.7,0.00,228,1,100.0,10
2010-01-28 14:50:00,27.7,28.7,27.0,32,9.5,0.0,WNW,0.00,16.1,NNW,...,0.065,29.4,30,10.1,28.4,0.00,224,1,98.2,10
2010-01-28 15:00:00,26.4,26.9,25.9,33,8.8,0.0,SE,0.00,16.1,NNW,...,0.056,28.3,30,9.1,27.2,0.33,227,1,99.6,10


In [13]:
data.info()

data.iloc[271020]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 606426 entries, 2010-01-28 14:20:00 to 2022-01-01 00:00:00
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Temp Out       606426 non-null  object 
 1   Hi Temp        606426 non-null  object 
 2   Low Temp       606426 non-null  object 
 3   Out Hum        606426 non-null  object 
 4   Dew Pt.        606426 non-null  object 
 5   Wind Speed     606426 non-null  float64
 6   Wind Dir       606426 non-null  object 
 7   Wind Run       606426 non-null  float64
 8   Hi Speed       606426 non-null  float64
 9   Hi Dir         606426 non-null  object 
 10  Wind Chill     606426 non-null  object 
 11  Heat Index     606426 non-null  object 
 12  THW Index      606426 non-null  object 
 13  THSW Index     606426 non-null  object 
 14  Bar            606426 non-null  object 
 15  Rain           606426 non-null  float64
 16  Rain Rate      606426 non-null  float64


Temp Out           ---
Hi Temp            ---
Low Temp           ---
Out Hum            ---
Dew Pt.            ---
Wind Speed         0.0
Wind Dir           WSW
Wind Run           0.0
Hi Speed           1.6
Hi Dir             WSW
Wind Chill         ---
Heat Index         ---
THW Index          ---
THSW Index         ---
Bar              760.9
Rain               0.0
Rain Rate          0.0
Solar Rad.           0
Solar Energy       0.0
Hi Solar Rad.        0
Heat D-D           ---
Cool D-D           ---
In Temp           22.3
In Hum              50
In Dew            11.4
In Heat           21.8
ET                 0.0
Wind Samp          230
Wind Tx              1
ISS Recept       100.0
Arc. Int            10
Name: 2015-04-25 21:30:00, dtype: object

### Agregamos los datos que faltan

In [9]:
newData = data.resample('10min', origin='start').interpolate()

In [10]:
newData.info()
newData.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 627179 entries, 2010-01-28 14:20:00 to 2022-01-01 00:00:00
Freq: 10T
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Temp Out       606423 non-null  object 
 1   Hi Temp        606423 non-null  object 
 2   Low Temp       606423 non-null  object 
 3   Out Hum        606423 non-null  object 
 4   Dew Pt.        606423 non-null  object 
 5   Wind Speed     627179 non-null  float64
 6   Wind Dir       606423 non-null  object 
 7   Wind Run       627179 non-null  float64
 8   Hi Speed       627179 non-null  float64
 9   Hi Dir         606423 non-null  object 
 10  Wind Chill     606423 non-null  object 
 11  Heat Index     606423 non-null  object 
 12  THW Index      606423 non-null  object 
 13  THSW Index     606423 non-null  object 
 14  Bar            606423 non-null  object 
 15  Rain           627179 non-null  float64
 16  Rain Rate      627179 non-null

,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Dir,Wind Run,Hi Speed,Hi Dir,...,Cool D-D,In Temp,In Hum,In Dew,In Heat,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int
Date Time,,,,,,,,,,,,,,,,,,,,,
2010-01-28 14:20:00,32.9,33.1,32.9,21,7.7,0.0,---,0.00,0.0,---,...,0.101,33.0,21,7.8,31.2,0.00,227.0,1.0,99.6,10.0
2010-01-28 14:30:00,32.9,33.1,32.7,24,9.7,1.6,ENE,0.27,41.8,NNE,...,0.101,33.2,24,9.9,31.8,0.00,229.0,1.0,100.0,10.0
2010-01-28 14:40:00,30.6,32.6,28.7,28,10.0,0.0,N,0.00,22.5,NNW,...,0.085,32.1,26,10.1,30.7,0.00,228.0,1.0,100.0,10.0
2010-01-28 14:50:00,27.7,28.7,27.0,32,9.5,0.0,WNW,0.00,16.1,NNW,...,0.065,29.4,30,10.1,28.4,0.00,224.0,1.0,98.2,10.0
2010-01-28 15:00:00,26.4,26.9,25.9,33,8.8,0.0,SE,0.00,16.1,NNW,...,0.056,28.3,30,9.1,27.2,0.33,227.0,1.0,99.6,10.0


### Convertimos estampillas de tiempo todo a segundos

In [7]:
timestamp_s = data.index
timestamp_s = timestamp_s.map(pd.Timestamp.timestamp)

day = 24*60*60
year = 365.2425 * day

data['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
data['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
data['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
data['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

## Correlación de los datos

In [8]:
data.corr()

,Wind Speed,Wind Run,Hi Speed,Rain,Rain Rate,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int,Day sin,Day cos,Year sin,Year cos
Wind Speed,1.000000,0.999991,0.918792,0.009120,0.005253,0.136506,0.026907,-0.001176,0.045139,NaN,-0.217967,-0.244458,-0.089430,-0.007408
Wind Run,0.999991,1.000000,0.919199,0.009105,0.005231,0.136722,0.027005,-0.001179,0.045255,NaN,-0.218190,-0.245083,-0.089461,-0.007255
Hi Speed,0.918792,0.919199,1.000000,0.004768,0.004215,0.160579,0.063346,-0.001555,0.074608,NaN,-0.244327,-0.274406,-0.093985,0.052207
Rain,0.009120,0.009105,0.004768,1.000000,0.514679,-0.002556,-0.005220,-0.000037,-0.003466,NaN,-0.006249,0.001829,0.009757,-0.014060
Rain Rate,0.005253,0.005231,0.004215,0.514679,1.000000,-0.001726,-0.003446,-0.000028,-0.002191,NaN,-0.002306,0.000558,0.008787,-0.007851
ET,0.136506,0.136722,0.160579,-0.002556,-0.001726,1.000000,0.040633,-0.000462,0.041172,NaN,-0.108423,-0.187189,-0.020894,0.072437
Wind Samp,0.026907,0.027005,0.063346,-0.005220,-0.003446,0.040633,1.000000,0.000492,0.991889,NaN,-0.101279,-0.016170,0.004238,0.051969
Wind Tx,-0.001176,-0.001179,-0.001555,-0.000037,-0.000028,-0.000462,0.000492,1.000000,0.000439,NaN,-0.000054,0.002566,0.001536,-0.002034
ISS Recept,0.045139,0.045255,0.074608,-0.003466,-0.002191,0.041172,0.991889,0.000439,1.000000,NaN,-0.100730,-0.011023,0.008692,0.054875
Arc. Int,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dropeamos las características que no tienen correlación

In [9]:
df = data.drop(dropped_features, axis = 1)

predicted_feature = df.columns.get_loc(predicted_feature)

KeyError: "['Tpot (K)' 'sh (g/kg)' 'H2OC (mmol/mol)'] not found in axis"

### Normalización de los datos

In [ ]:
df_length = len(df)

train_length = int(df_length*train_perc)
val_length   = int(df_length*(train_perc+val_perc))

train_mean = df.values[:train_length].mean(axis=0)
train_std  = df.values[:train_length].std(axis=0)

df_norm = (df.values - train_mean) / train_std
df_norm = pd.DataFrame(df_norm)

### Dividimos los datos


In [ ]:
train_df = df_norm[:train_length-1]
val_df   = df_norm[train_length:val_length-1]
test_df_norm  = df_norm[val_length:]
#test_df       = df[val_length:]

###  Create datasets


In [ ]:
def create_y_lento(df_norm, start, end, length, min_temp):
    y_data = []
    
    for i in range(start, end):
        freeze = 0
        for j in range(i, i+length):
            if df_norm.iloc[j][predicted_feature] <= min_temp:
                freeze = 1
                break
        y_data.append(freeze)

    return y_data

In [ ]:
def create_y(df, start, end, length, min_temp):
    y_data = []
    
    df_bool = df['T (degC)'][start:end] <= min_temp
    
    for i in range(start, end):
        freeze = 0
        if np.any(df_bool[i:i+length]):
            freeze = 1
        y_data.append(freeze)

    return y_data


In [ ]:
start = sequence_length*sampling_rate + offset*sampling_rate
end   = train_length + offset*sampling_rate

x_train = train_df
y_train = create_y(df, start, end, length*6, min_temp)
#df_norm[[predicted_feature]][start:end]

dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train.values,
    y_train,
    sequence_length = sequence_length,
    sampling_rate = sampling_rate,
    shuffle=True,
    batch_size = batch_size
)

In [ ]:
print((end-start))
tf.data.experimental.cardinality(dataset_train)
print(dataset_train)

print(y_train)

In [ ]:
start = train_length + sequence_length*sampling_rate + offset*sampling_rate
end   = val_length + offset*sampling_rate

x_val = val_df
y_val = create_y(df, start, end, length*6, min_temp)
#df_norm[[predicted_feature]][start:end]

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val.values,
    y_val,
    sequence_length = sequence_length,
    sampling_rate = sampling_rate,
    shuffle=False,
    batch_size = batch_size
)

In [ ]:
print(y_val)

In [ ]:
start = val_length + sequence_length*sampling_rate + offset*sampling_rate
x_end = len(test_df_norm) - sequence_length*sampling_rate - offset*sampling_rate

x_test = test_df_norm[:x_end]
y_test = create_y(df, start, len(df_norm), length*6, min_temp)#df_norm[[predicted_feature]][start:]

dataset_test = keras.preprocessing.timeseries_dataset_from_array(
    x_test.values,
    y_test,
    sequence_length = sequence_length,
    sequence_stride = sampling_rate,
    sampling_rate = sampling_rate,
    shuffle=False,
    batch_size = batch_size
)

In [ ]:
print(y_test)

### Creamos el modelo


In [ ]:
for batch in dataset_train.take(1):
    x, y = batch
    
input_shape = x.shape[1], x.shape[2]

In [ ]:
inputs = keras.layers.Input(input_shape)
lstm_layer = keras.layers.LSTM(32, return_sequences=True)(inputs)
lstm_layer2 = keras.layers.LSTM(32)(lstm_layer)
output = keras.layers.Dense(1)(lstm_layer2)

model = keras.Model(inputs, output)
model.compile(keras.optimizers.Adam(learning_rate), loss)
model.summary()

### Entrenamos el modelo

In [ ]:
path_checkpoint = "model_test.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_best_only=True,
)

history = model.fit(dataset_train, epochs = epochs, validation_data=dataset_val, callbacks = [es_callback, modelckpt_callback])

In [ ]:
for batch in dataset_train.take(1):
    x, y = batch
    
print(x.shape)

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

visualize_loss(history, "Training and Validation Loss")

### Mostramos los resultados

In [ ]:
for batch in dataset_test.take(5):
    x, y = batch
    y_pred = model.predict(x)

    #y_pred = y_pred * train_std[predicted_feature] + train_mean[predicted_feature]
    #y = y * train_std[predicted_feature] + train_mean[predicted_feature]
    
    fig, ax = plt.subplots()
    
    ax.plot(y[0:100000], c='r', label="Test Data")
    ax.plot(y_pred[0:100000],c='g', label="Prediction")
    plt.legend()
    plt.show()

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return


for x1, y1 in dataset_test.take(10):
    show_plot(
        [x1[0][:, 1].numpy(), y1[0].numpy(), model.predict(x1)[0]],
        6,
        "Single Step Prediction",
    )